<a href="https://colab.research.google.com/github/Prachii26/Autogluon/blob/main/colabs/Autogluon%20Kaggle_IEEE_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Autogluon-IEEE-CIS Fraud Detection
##Submitted by- Prachi Gupta
##SJSUID- 019106594

In [2]:
!pip -q install -U pip setuptools wheel
!pip -q install -U autogluon.tabular

# (optional) print versions for reproducibility
import importlib.metadata as im, sys, platform
print("AutoGluon Tabular:", im.version('autogluon.tabular'))
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())

AutoGluon Tabular: 1.4.0
Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35


In [4]:
# Configure Kaggle credentials
import json, os, pathlib
p = pathlib.Path.home() / ".kaggle"
p.mkdir(exist_ok=True)
with open("/content/kaggle.json") as f:
    data = json.load(f)
with open(p/"kaggle.json","w") as f:
    json.dump(data, f)
os.chmod(p/"kaggle.json", 0o600)

# Install the Kaggle CLI
!pip -q install kaggle
!kaggle -v

/usr/local/lib/python3.12/dist-packages/google/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Kaggle API 1.7.4.5


In [5]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

# Download the competition files (≈800MB compressed)
!kaggle competitions download -c ieee-fraud-detection -p /content/ieee
!unzip -o /content/ieee/ieee-fraud-detection.zip -d /content/ieee > /dev/null

directory="/content/ieee"
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path =  '/content/ieee/AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'/train_identity.csv')
train_transaction = pd.read_csv(directory+'/train_transaction.csv')

# List files so we know what we have
!ls -lh /content/ieee | sed -n '1,200p'


/usr/local/lib/python3.12/dist-packages/google/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
  0% 0.00/118M [00:00<?, ?B/s]
100% 118M/118M [00:00<00:00, 3.15GB/s]
total 1.4G
-rw-r--r-- 1 root root 119M Dec 11  2019 ieee-fraud-detection.zip
-rw-r--r-- 1 root root 5.8M Dec 11  2019 sample_submission.csv
-rw-r--r-- 1 root root  25M Dec 11  2019 test_identity.csv
-rw-r--r-- 1 root root 585M Dec 11  2019 test_transaction.csv
-rw-r--r-- 1 root root  26M Dec 11  2019 train_identity.csv
-rw-r--r-- 1 root root 652M Dec 11  2019 train_transaction.csv


In [6]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [10]:
!pip install autogluon.tabular[lightgbm]==1.4.0

predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='high', time_limit=1800
)

results = predictor.fit_summary()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 106.0 MB/s  0:00:00


Preset alias specified: 'high' maps to 'high_quality'.
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          4
GPU Count:          0
Memory Avail:       23.80 GB / 47.05 GB (50.6%)
Disk Space Avail:   200.15 GB / 225.33 GB (88.8%)
Presets specified: ['high']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'num_bag_sets': 1,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
     

[50]	valid_set's binary_logloss: 0.0947332
[100]	valid_set's binary_logloss: 0.0856834
[150]	valid_set's binary_logloss: 0.0811864
[200]	valid_set's binary_logloss: 0.0779008
[250]	valid_set's binary_logloss: 0.075393
[300]	valid_set's binary_logloss: 0.0733887
[350]	valid_set's binary_logloss: 0.0715076
[400]	valid_set's binary_logloss: 0.0701123
[450]	valid_set's binary_logloss: 0.068894


	Ran out of time, early stopping on iteration 503. Best iteration is:
	[503]	valid_set's binary_logloss: 0.0677429


[500]	valid_set's binary_logloss: 0.0677996


...

	Ran out of time, early stopping on iteration 1014. Best iteration is:
	[1012]	valid_set's binary_logloss: 0.0469304
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}


[50]	valid_set's binary_logloss: 0.0625507
[100]	valid_set's binary_logloss: 0.0557464
[150]	valid_set's binary_logloss: 0.05356
[200]	valid_set's binary_logloss: 0.0520523
[250]	valid_set's binary_logloss: 0.05091
[300]	valid_set's binary_logloss: 0.0504181
[350]	valid_set's binary_logloss: 0.0499344
[400]	valid_set's binary_logloss: 0.0495987
[450]	valid_set's binary_logloss: 0.0489282
[500]	valid_set's binary_logloss: 0.0486242
[550]	valid_set's binary_logloss: 0.0483723
[600]	valid_set's binary_logloss: 0.0482839
[650]	valid_set's binary_logloss: 0.0481091
[700]	valid_set's binary_logloss: 0.0480042
[750]	valid_set's binary_logloss: 0.047863
[800]	valid_set's binary_logloss: 0.0476622
[850]	valid_set's binary_logloss: 0.0475751
[900]	valid_set's binary_logloss: 0.047384
[950]	valid_set's binary_logloss: 0.0472703
[1000]	valid_set's binary_logloss: 0.0470939
[1050]	valid_set's binary_logloss: 0.0469735
[1100]	valid_set's binary_logloss: 0.0469074
[1150]	valid_set's binary_logloss: 0

	Ran out of time, early stopping on iteration 1269. Best iteration is:
	[1269]	valid_set's binary_logloss: 0.0466797
Saving /content/ieee/AutoGluonModels/models/LightGBMXT_BAG_L2/utils/oof.pkl
Saving /content/ieee/AutoGluonModels/models/LightGBMXT_BAG_L2/model.pkl
	0.967	 = Validation score   (roc_auc)
	388.37s	 = Training   runtime
	10.94s	 = Validation runtime
	1977.6	 = Inference  throughput (rows/s | 73818 batch size)
Saving /content/ieee/AutoGluonModels/models/trainer.pkl
Fitting model: LightGBM_BAG_L2 ... Training model for up to 13.15s of the 12.87s of remaining time.
	Fitting LightGBM_BAG_L2 with 'num_gpus': 0, 'num_cpus': 4
Saving /content/ieee/AutoGluonModels/models/LightGBM_BAG_L2/utils/model_template.pkl
Loading: /content/ieee/AutoGluonModels/models/LightGBM_BAG_L2/utils/model_template.pkl
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=4, gpus=0)
	Time limit exceeded... Skipping LightGBM_BAG_L2.
Loading: /content/ie

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.966967     roc_auc      37.417952  1174.629121                0.091485           4.544924            3      False          4
1         LightGBMXT_BAG_L2   0.966962     roc_auc      37.326466  1170.084197               10.940934         388.370572            2      False          3
2         LightGBMXT_BAG_L1   0.962427     roc_auc      26.385533   781.713625               26.385533         781.713625            1      False          1
3       WeightedEnsemble_L2   0.962427     roc_auc      26.478680   781.809559                0.093147           0.095934            2      False          2
4  WeightedEnsemble_L3_FULL        NaN     roc_auc            NaN   163.719201                     NaN           4.544924            3       True        

/usr/local/lib/python3.12/dist-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [12]:
test_identity = pd.read_csv(directory+'/test_identity.csv')
test_transaction = pd.read_csv(directory+'/test_transaction.csv')

# Identify missing 'id_' columns in test_identity compared to train_identity
missing_id_cols = [col for col in train_identity.columns if col.startswith('id_') and col not in test_identity.columns]

# Add missing 'id_' columns to test_identity with NaN values
for col in missing_id_cols:
    test_identity[col] = np.nan

test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

Loading: /content/ieee/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/ieee/AutoGluonModels/models/LightGBMXT_BAG_L2_FULL/model.pkl
Loading: /content/ieee/AutoGluonModels/models/WeightedEnsemble_L3_FULL/model.pkl


,0,1
0,0.999099,0.000901
1,0.999935,0.000065
2,0.999846,0.000155
3,0.999683,0.000317
4,0.999933,0.000066


In [13]:
predictor.positive_class

1

In [14]:
predictor.class_labels

[0, 1]

In [15]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

Loading: /content/ieee/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/ieee/AutoGluonModels/models/LightGBMXT_BAG_L2_FULL/model.pkl
Loading: /content/ieee/AutoGluonModels/models/WeightedEnsemble_L3_FULL/model.pkl


In [17]:
submission = pd.read_csv(directory+'/sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'/my_submission.csv', index=False)